In [1]:
from qiskit import IBMQ
# IBMQ.save_account(MY_API_TOKEN)
import qiskit 
qiskit.__version__

import scipy
import numpy as np
import random
from sklearn import preprocessing
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info.operators import Operator
from qiskit import(QuantumCircuit, execute, Aer)
from qiskit.visualization import plot_histogram
from qiskit.extensions import Initialize # Import the Inititialize function
from qiskit.aqua.circuits.gates import multi_control_toffoli_gate
from qiskit.aqua.circuits.gates import multi_control_multi_target_gate

In [64]:
#definicja rozmiaru sieci
visible = 1
hidden = 1
ancilla = visible-1

#definicja wejścia (x) oraz inicjalizacja macierzy wag
x = np.array([random.uniform(0, 1) for n in range(visible)])
weight_matrix = np.random.rand(visible, hidden) * np.pi

In [65]:
# definicja obwodu (linii qubitów)
# qc = QuantumCircuit(visible + hidden, visible)

qr = QuantumRegister(visible + hidden + ancilla, 'q')
cr = ClassicalRegister(visible + hidden, 'c')
qc = QuantumCircuit(qr, cr)

# inicjalizacja wartości qubitów wejściowych (x) na razie randomowa
initial_state = [[np.sqrt(x[i]), np.sqrt(1-x[i])] for i in range(len(x))]

# inicjalizacja wartości qubitów wejściowych i bramka Hadamarda
for i in range(visible):
    initialize_qubit = Initialize(initial_state[i])
    qc.append(initialize_qubit, [i])
    qc.h(i)
    
# ciąg bramek CNOT i bramek rotacji R (zależnych od parametrów)
for i in range(hidden):
    for j in range(visible):
        qc.rz(weight_matrix[j][i], j)
    multi_control_toffoli_gate.mct(qc, [qr[k] for k in range(visible)], qr[visible + i], [qr[i] for i in range(visible + hidden, visible + hidden + ancilla)], mode='basic')
    
    
    
# bramki Hadamarda na warstwie ukrytej
for i in range(hidden):
    qc.h(visible + i)

    
# pomiar linii visible
qc.measure(list(range(visible, visible+hidden)), list(range(visible, visible+hidden)))


### OBWÓD DRUGĄ STRONĘ

# wyzerowanie wartości qubitów visible
for i in range(visible):
    initialize_qubit = Initialize([1, 0])
    qc.append(initialize_qubit, [i])
    
# bramka Hadamarda na qubitach hidden
for i in range(visible, hidden + visible):
    qc.h(i)
    
#odwrócone rotacje i CNOTy
for i in range(hidden):
    for j in range(visible):
        multi_control_toffoli_gate.mct(qc, [qr[k] for k in range(visible, visible + hidden)], qr[j], [qr[k] for k in range(visible + hidden, visible + hidden + ancilla)], mode='basic')
        qc.rz(-weight_matrix[j][hidden - 1 - i], j)
        
# bramka Hadamarda i pomiar na qubitach visible
for i in range(visible):
    qc.h(i)
    qc.measure(list(range(visible)), list(range(visible)))
    

In [66]:
# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator
job = execute(qc, simulator, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(qc)
print("\nTotal count for 0 and 1 are:",counts)
# qc.draw(output='mpl')
qc.draw()


Total count for 0 and 1 are: {'11': 252, '10': 262, '00': 248, '01': 238}


┌─────────────────────────────┐┌───┐┌────────────┐     »
q_0: |0>┤ Initialize(0.73918,0.67351) ├┤ H ├┤ Rz(2.6096) ├──■──»
        └─────────────────────────────┘└───┘└────────────┘┌─┴─┐»
q_1: |0>──────────────────────────────────────────────────┤ X ├»
                                                          └───┘»
 c_0: 0 ═══════════════════════════════════════════════════════»
                                                               »
 c_1: 0 ═══════════════════════════════════════════════════════»
                                                               »
«     ┌─────────────────┐        ┌───┐┌─────────────┐┌───┐┌─┐
«q_0: ┤ Initialize(1,0) ├────────┤ X ├┤ Rz(-2.6096) ├┤ H ├┤M├
«     └──────┬───┬──────┘┌─┐┌───┐└─┬─┘└─────────────┘└───┘└╥┘
«q_1: ───────┤ H ├───────┤M├┤ H ├──■───────────────────────╫─
«            └───┘       └╥┘└───┘                          ║ 
«c_0: ════════════════════╬════════════════════════════════╩═
«                         ║                                  
«c_1: ════════════════════╩══════════════════════════════════
«

In [38]:
# komórka do testowania wieloqubitowej bramki CNOT
# NIE ZALEŻNA OD WŁAŚCIWEJ SIECI

# work = ancilla
def n_control_gate(qc, target, control: list, work: list, gate_type):
    if len(control) - len(work) != 1:
        raise Exception("Wrong number of control or work qubits!")

    qc.toffoli(control[0], control[1], work[0])
    control_left = control[2:]
    
    for i, c_bit in enumerate(control_left):
        qc.toffoli(c_bit, work[i], work[i+1])
    
    qc.gate_type(work[-1], target)
    
    for i, c_bit in reversed(list(enumerate(control_left))):
        qc.toffoli(c_bit, work[i], work[i+1])
    
    qc.toffoli(control[0], control[1], work[0])
    
    

        

visible = 2
hidden = 2
ancilla = visible - 1

# qr_visible = QuantumRegister(visible, 'visible')
# qr_hidden = QuantumRegister(hidden, 'hidden')
# cr = ClassicalRegister(3, 'c')
# if visible > 2:
#     anc = QuantumRegister(visible - 2, 'ancilla')
#     qc = QuantumCircuit(qr_visible, qr_hidden, anc, cr)
# else:
#     qc = QuantumCircuit(qr_visible, qr_hidden, cr)


# qr_visible = QuantumRegister(visible, 'visible')
qr = QuantumRegister(visible + hidden + ancilla, 'q')
cr = ClassicalRegister(3, 'c')
qc = QuantumCircuit(qr, cr)


for i in range(visible + hidden):
    initialize_qubit = Initialize([0, 1])
    qc.append(initialize_qubit, [i])
# initialize_qubit = Initialize([1, 0])
# qc.append(initialize_qubit, [2])
# for i in range(hidden):
#     for j in range(visible):
#         qc.rz(weight_matrix[j][i], j)
print(list(range(visible)) + [visible + 0])
print(list(range(visible+hidden, visible+hidden+ancilla)))
# multi_control_multi_target_gate.mcmt(qc, list(range(visible)), [], QuantumCircuit.cx, [visible])

# print([qr_visible[i] for i in range(visible)])
# print([anc[i] for i in range(ancilla)])
# print([qr_hidden[i] for i in range(hidden)])
for j in range(hidden):
    multi_control_multi_target_gate.mcmt(qc, [qr[i] for i in range(visible)], [qr[i] for i in range(visible + hidden, visible + hidden + ancilla)], QuantumCircuit.cx, [qr[visible + j]])

# multi_control_multi_target_gate.mcmt(qc, [qr_visible[i] for i in range(visible)], [anc[i] for i in range(ancilla)], QuantumCircuit.cx, [qr_hidden[0]])
# qc.measure(list(range(visible+hidden)), list(range(visible + hidden)))
# (qc, list(range(visible)) + [visible + 0], qancilla = 6)


# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator
job = execute(qc, simulator, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(qc)
print("\nTotal count for 0 and 1 are:",counts)
qc.draw()

No measurements in circuit "circuit230": count data will return all zeros.


[0, 1, 2]
[4]

Total count for 0 and 1 are: {'000': 1000}


┌─────────────────┐                              
q_0: |0>┤ Initialize(0,1) ├──■─────────■────■─────────■──
        ├─────────────────┤  │         │    │         │  
q_1: |0>┤ Initialize(0,1) ├──■─────────■────■─────────■──
        ├─────────────────┤  │  ┌───┐  │    │         │  
q_2: |0>┤ Initialize(0,1) ├──┼──┤ X ├──┼────┼─────────┼──
        ├─────────────────┤  │  └─┬─┘  │    │  ┌───┐  │  
q_3: |0>┤ Initialize(0,1) ├──┼────┼────┼────┼──┤ X ├──┼──
        └─────────────────┘┌─┴─┐  │  ┌─┴─┐┌─┴─┐└─┬─┘┌─┴─┐
q_4: |0>───────────────────┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├
                           └───┘     └───┘└───┘     └───┘
 c_0: 0 ═════════════════════════════════════════════════
                                                         
 c_1: 0 ═════════════════════════════════════════════════
                                                         
 c_2: 0 ═════════════════════════════════════════════════

In [6]:
print(weight_matrix, '\n')
sinus_matrix = np.sin(weight_matrix + np.pi/4)
print(sinus_matrix, '\n')
print(sinus_matrix.prod(axis = 0), '\n')
phi = np.arcsin(sinus_matrix.prod(axis = 0))
print(phi, '\n')
psi = phi + np.pi/4
print(psi, '\n')
ph = np.sin(psi)**2
print(ph, '\n')
# ph_normalized = preprocessing.normalize([ph], norm = 'l1')[0]
# print(ph_normalized)

[[0.81414165 1.71981054]
 [1.11110563 1.20259205]] 

[[0.99958693 0.59429113]
 [0.94742459 0.91422957]] 

[0.94703324 0.54331852] 

[1.24386802 0.57438492] 

[2.02926619 1.35978308] 

[0.80412604 0.95613037] 



In [89]:
def exctract_single_qubit_measurment(dict_of_counts, qubit_range):
    num_qubits = int(np.log2(len(counts)))
#     result = np.zeros(len(qubit_range))
    result = np.zeros(num_qubits)
    for el in dict_of_counts:
        for i in range(num_qubits):
#             print("i", i)
#             print("el[i]", el[i])
            if i in qubit_range and el[i] == '1':
                result[i] += dict_of_counts[el]
#     print(result)
#     print(result[qubit_range])
    return result[qubit_range]

In [91]:
# template do uczenia

def update(exp_ph):
    global weight_matrix
    #obliczanie prawdopodobieństwa p(|h>) ze wzoru
    sinus_matrix = np.sin(weight_matrix + np.pi/4)
    phi = np.arcsin(sinus_matrix.prod(axis = 0))
#     print('phi', phi)
    psi = phi + np.pi/4
#     print('psi', psi)
    ph = np.sin(psi)**2
#     print('ph', ph)
    
    #obliczanie gradientu i,k
    S = 1/(np.sqrt(1-(np.sin(phi))**2))
#     print("S", S)
    T = np.sin(phi) * np.sum(1/np.tan(weight_matrix + np.pi/4), axis = 0)
#     print("T", T)
#     print("sum products", (exp_ph - ph) * ph * np.cos(2*psi) * S * T)
    gradient = np.zeros(hidden)
    for k in range(hidden):
        gradient[k] = np.sum( (exp_ph - ph) * ph * np.cos(2*psi) * S * T[k])
#     print("gradient = ", gradient)
#     print([gradient] * len(weight_matrix))
    
    weight_matrix = weight_matrix + ([gradient] * len(weight_matrix))


    
#definicja rozmiaru sieci
visible = 3
hidden = 2
ancilla = visible-1

#definicja wejścia (x)oraz inicjalizacja macierzy wag
# x = np.array([random.uniform(0, 1) for n in range(visible)])
x = np.array([0.25 for n in range(visible)])
weight_matrix = np.random.rand(visible, hidden) * np.pi

#definicja parametrów uczenia
num_shots = 1000
num_epochs = 100

for epoch in range(num_epochs):
    ##########################
    # definicja obwodu (linii qubitów)
    # qc = QuantumCircuit(visible + hidden, visible)

    qr = QuantumRegister(visible + hidden + ancilla, 'q')
    cr = ClassicalRegister(visible + hidden, 'c')
    qc = QuantumCircuit(qr, cr)

    # inicjalizacja wartości qubitów wejściowych (x) na razie randomowa
    initial_state = [[np.sqrt(x[i]), np.sqrt(1-x[i])] for i in range(len(x))]

    # inicjalizacja wartości qubitów wejściowych i bramka Hadamarda
    for i in range(visible):
        initialize_qubit = Initialize(initial_state[i])
        qc.append(initialize_qubit, [i])
        qc.h(i)

    # ciąg bramek CNOT i bramek rotacji R (zależnych od parametrów)
    for i in range(hidden):
        for j in range(visible):
            qc.rz(weight_matrix[j][i], j)
        multi_control_toffoli_gate.mct(qc, [qr[k] for k in range(visible)], qr[visible + i], [qr[i] for i in range(visible + hidden, visible + hidden + ancilla)], mode='basic')



    # bramki Hadamarda na warstwie ukrytej
    for i in range(hidden):
        qc.h(visible + i)


    # pomiar linii visible
    qc.measure(list(range(visible, visible+hidden)), list(range(visible, visible+hidden)))


    ### OBWÓD DRUGĄ STRONĘ

    # wyzerowanie wartości qubitów visible
    for i in range(visible):
        initialize_qubit = Initialize([1, 0])
        qc.append(initialize_qubit, [i])

    # bramka Hadamarda na qubitach hidden
    for i in range(visible, hidden + visible):
        qc.h(i)

    #odwrócone rotacje i CNOTy
    for i in range(hidden):
        for j in range(visible):
            multi_control_toffoli_gate.mct(qc, [qr[k] for k in range(visible, visible + hidden)], qr[j], [qr[k] for k in range(visible + hidden, visible + hidden + ancilla)], mode='basic')
            qc.rz(-weight_matrix[j][hidden - 1 - i], j)

    # bramka Hadamarda i pomiar na qubitach visible
    for i in range(visible):
        qc.h(i)
        qc.measure(list(range(visible)), list(range(visible)))
    
    
    #################################
    #eksperyment:
    #symylator
    simulator = Aer.get_backend('qasm_simulator')

    # Execute the circuit on the qasm simulator
    job = execute(qc, simulator, shots=num_shots)

    # Grab results from the job
    result = job.result()

    # Returns counts
    counts = result.get_counts(qc)
    exp_ph = exctract_single_qubit_measurment(counts, list(range(visible, visible + hidden))) / num_shots
#     print("\nTotal count for hidden qubits '1' measurments are:",exctract_single_qubit_measurment(counts))
    print("\nProbabilities are:",exp_ph)
#     print("\nTotal count for 0 and 1 are:",counts)

    print(weight_matrix)
    qc.draw()    
    
    update(exp_ph)
    


Probabilities are: [0.517 0.525]
[[1.97331488 3.14045789]
 [1.31677121 1.36912852]
 [2.72154589 1.31136893]]

Probabilities are: [0.511 0.508]
[[1.97545297 3.14541158]
 [1.31890929 1.37408221]
 [2.72368397 1.31632262]]

Probabilities are: [0.495 0.483]
[[1.97766693 3.15064739]
 [1.32112326 1.37931802]
 [2.72589794 1.32155843]]

Probabilities are: [0.512 0.488]
[[1.97986547 3.15594993]
 [1.32332179 1.38462056]
 [2.72809647 1.32686097]]

Probabilities are: [0.511 0.478]
[[1.98233531 3.16201592]
 [1.32579163 1.39068655]
 [2.73056631 1.33292696]]

Probabilities are: [0.512 0.479]
[[1.98493662 3.16852631]
 [1.32839295 1.39719694]
 [2.73316763 1.33943735]]

Probabilities are: [0.499 0.501]
[[1.98773857 3.17566843]
 [1.3311949  1.40433906]
 [2.73596958 1.34657947]]

Probabilities are: [0.483 0.498]
[[1.99076999 3.18353694]
 [1.33422632 1.41220756]
 [2.73900099 1.35444797]]

Probabilities are: [0.494 0.513]
[[1.99389495 3.19179535]
 [1.33735128 1.42046598]
 [2.74212595 1.36270639]]

Probabili


Probabilities are: [0.513 0.502]
[[2.32114696 3.92208574]
 [1.66460329 2.15075637]
 [3.06937797 2.09299678]]

Probabilities are: [0.504 0.483]
[[2.32240086 3.92344064]
 [1.66585719 2.15211127]
 [3.07063186 2.09435167]]

Probabilities are: [0.496 0.512]
[[2.32319888 3.92429836]
 [1.66665521 2.15296899]
 [3.07142988 2.0952094 ]]

Probabilities are: [0.487 0.481]
[[2.32450139 3.92569361]
 [1.66795772 2.15436424]
 [3.07273239 2.09660465]]

Probabilities are: [0.506 0.475]
[[2.32512735 3.92636047]
 [1.66858368 2.1550311 ]
 [3.07335836 2.09727151]]

Probabilities are: [0.522 0.474]
[[2.32572813 3.92699881]
 [1.66918446 2.15566944]
 [3.07395913 2.09790984]]

Probabilities are: [0.482 0.487]
[[2.32638159 3.92769137]
 [1.66983792 2.156362  ]
 [3.07461259 2.0986024 ]]

Probabilities are: [0.521 0.5  ]
[[2.32706589 3.92841463]
 [1.67052222 2.15708525]
 [3.07529689 2.09932566]]

Probabilities are: [0.479 0.495]
[[2.32817817 3.92958686]
 [1.6716345  2.15825749]
 [3.07640917 2.1004979 ]]

Probabili